# BART Webinar: The `reconet` Command

This tutorial uses the [BART](http://mrirecon.github.io/bart/) command-line inteface (CLI) and presents how to train and apply neural networks for image reconstruction using BART.

The `bart reconet` command implements physics based reconstruction networks solving the inverse SENSE problem.
The SENSE operator is generically implemented. For example, it supports multiple sets of coil sensitivity maps or non-Cartesian trajectories.
Proximal mappings and gradient steps can be used as data-consistency terms such that `reconet` can be used for example to train/infer the Variational Network<sup>1</sup> or MoDL<sup>2</sup>.

The basic structure of the ```bart reconet``` command reads:
```bash
$ bart reconet --network=varnet --train <kspace> <coils> <weights> <reference>
$ bart reconet --network=varnet --apply <kspace> <coils> <weights> <reconstruction>
```

In this notebook, we present a self-contained example, how to train the Variational Network an MoDL on the NYU machine learning dataset<sup>1</sup> available at [mridata.org](http://mridata.org/), i.e. we preprocess the k-space data and estimate coil sensitivity maps using ESPIRiT.

For more information on the implementation details, we refer to out preprint [Deep, Deep Learning with BART](https://arxiv.org/abs/2202.14005).


**Author**: [Moritz Blumenthal](mailto:moritz.blumenthal@med.uni-goettingen.de)

**Presenter**: [Moritz Blumenthal](mailto:moritz.blumenthal@med.uni-goettingen.de)

**Institution**: University Medical Center Göttingen


1: Hammernik, K. et al. (2018), [Learning a variational network for reconstruction of accelerated MRI data](https://doi.org/10.1002/mrm.26977). Magn. Reson. Med., 79: 3055-3071.

2: Aggarwal, H. K. et al.(2019), [MoDL: Model-Based Deep Learning Architecture for Inverse Problems](https://doi.org/10.1109/TMI.2018.2865356). IEEE Trans. Med. Imag., 38(2): 394-405

## 1 General Remarks and Early Setup

This notebook is designed to run on a local system and on Google Colab. It uses the python kernel, however, allmost all comands use the `%%bash` cell magic to be executed in a `bash` subshell. If live output of a bash command is desired, we use the exclamation mark `!` to execute single lines.
`bash` environment variables are set in python to be shared across all following cells.

### 1.1 Google Colab

To run BART on Google Colab, this notebook automatically installs dependencies and setup the GPUs if the environment variable `COLAB=1`is set. If you run this notebook on your local system, you might not want this setup. Please set `COLAB=0`in this case.

For a detailed explanation, see the [How to Run BART on Google Colaboratory](https://github.com/mrirecon/bart-workshop/tree/master/ismrm2021).

This tutorial needs a GPU instance:
- Go to Edit → Notebook Settings
- Choose GPU from Hardware Accelerator drop-down menu

In [ ]:
%env COLAB=1

env: COLAB=1


Not all GPUs on Google Colab support CUDA 11, we downgrade CUDA if necessary:

In [ ]:
%%bash

[ $COLAB -ne 1 ] && echo "Skipp cell (not on Colab)" && exit 0

# Use CUDA 10.1 when on Tesla K80

# Estimate GPU Type
GPU_NAME=$(nvidia-smi --query-gpu=gpu_name --format=csv,noheader)

echo "GPU Type:"
echo $GPU_NAME

if [ "Tesla K80" = "$GPU_NAME" ];
then
    echo "GPU type Tesla K80 does not support CUDA 11. Set CUDA to version 10.1."

    # Change default CUDA to version 10.1
    cd /usr/local
    rm cuda
    ln -s cuda-10.1 cuda
else
    echo "Current GPU supports default CUDA-11."
    echo "No further actions are necessary."
fi

echo "GPU Information:"
nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv
nvcc --version

GPU Type:
Tesla T4
Current GPU supports default CUDA-11.
No further actions are necessary.
GPU Information:
name, driver_version, memory.total [MiB]
Tesla T4, 460.32.03, 15109 MiB
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


To have the same experience as if this notebook was checked cloned with git, we clone the BART-Webinars repository and change the current working directory to the `webinar6` directory:

In [ ]:
%%bash

[ $COLAB -ne 1 ] && echo "Skipp cell (not on Colab)" && exit 0

[ -d bart-webinars ] && rm -r bart-webinars
git clone https://github.com/mrirecon/bart-webinars.git > /dev/null

Cloning into 'bart-webinars'...
Checking out files: 100% (110/110), done.


In [ ]:
import os
pwd=os.getcwd()
if os.environ["COLAB"] == "1":
    pwd=pwd+"/bart-webinars/webinar6/demo1_reconet/"

%cd $pwd

/content/bart-webinars/webinar6


### 1.2 Demonstration Mode

Since training neural takes too long time for demonstration purposes, we skipp training in the demonstration mode by setting the environment variable `DEMO=1`.
If you want to reproduce the training, set `DEMO=0`, however, be warned that the storage on Google Colab might not be sufficient to hold and preprocess all training data and that the Colab runtime might disconnect due to the long training time.

In this notebook five networks are trained, each takes about 1h on an NVIDIA A100 GPU but can take much more time on older GPUs. Choose wisely, if you want to retrain them.

In [ ]:
%env DEMO=1

env: DEMO=1


Please note that the DEMO mode assumes that your current working directory is `webinar6` in the bart-webinars directory. You can change the current working directory persistently using `%cd bart-webinars/webinar6`. 

In [ ]:
%cd ./

/content/bart-webinars/webinar6


# Setup BART

### 2.1 Install libraries

We install dependencies for BART. Make sure that you have installed the requirements if you run locally:

In [ ]:
%%bash

[ $COLAB -ne 1 ] && echo "Skipp cell (not on Colab)" && exit 0

# Install BARTs dependencies
apt-get install -y make gcc libfftw3-dev liblapacke-dev libpng-dev libopenblas-dev &> /dev/null

# Install additional dependencies for converting ISMRMRD files
apt-get install -y libismrmrd-dev libboost-all-dev libhdf5-serial-dev &> /dev/null



### 2.2 Clone and Compile BART

We clone BART into the current working directory of this notebook and delete any previous installation in this directory.

In [ ]:
%%bash

# Clone Bart
[ -d bart ] && rm -r bart
git clone https://github.com/mrirecon/bart/ bart &> /dev/null

In [ ]:
%%bash

cd bart

# Define compile options
COMPILE_SPECS=" PARALLEL=1
                CUDA=1
                NON_DETERMINISTIC=1
                ISMRMRD=1
                "

printf "%s\n" $COMPILE_SPECS > Makefiles/Makefile.local


if [ $COLAB -eq "1" ]
then
    # set path to cuda for Colab
    echo "CUDA_BASE=/usr/local/cuda" >> Makefiles/Makefile.local
    echo "CUDA_LIB=lib64" >> Makefiles/Makefile.local
fi


make &> /dev/null

### 2.3 Add BART to PATH variable

We add the BART directory to the PATH variable and include the python wrapper for reading *.cfl files:

In [ ]:
import os
import sys

os.environ['TOOLBOX_PATH']=os.getcwd()+"/bart/"
os.environ['PATH'] = os.environ['TOOLBOX_PATH'] + ":" + os.environ['PATH']
sys.path.append(os.environ['TOOLBOX_PATH'] + "/python/")

Check BART setup:

In [ ]:
%%bash

echo "# The BART used in this notebook:"
which bart
echo "# BART version: "
bart version

# The BART used in this notebook:
/content/bart-webinars/webinar6/bart//bart
# BART version: 
v0.7.00-357-gcf0baa7


### 2.4 Interactive CFL-Viewer

For visualization of reconstructions etc., we define an interactive viewer for *.cfl files based on ipython widgts.

Usage: plot(["file1", "file2"])

In [ ]:
%matplotlib inline

def plot(files, title=None):
  import numpy as np
  from matplotlib import pyplot as plt
  import cfl
  import os

  from ipywidgets import interact, interactive, fixed, interact_manual
  import ipywidgets as widgets

  def update(Range=(0.,1.),Coil=0, Map=0, Slice=0, Batch=0):

    if(title==None):
      wtitle=[]
      for file in files:
        head, tail = os.path.split(file)
        wtitle.append(tail)
    else:
      wtitle=title
    
    data=cfl.readcfl(files[0])
    rat=data.shape[0]/data.shape[1]
    width=16
    
    ncols=len(files)
    nrows=1
    
    rat=rat
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, constrained_layout=True, squeeze=False, figsize=(width, width*rat))

    for i in range(len(files)):
 
        data=cfl.readcfl(files[i])
        nshp = [1] * 16

        for k in range(len(data.shape)):
          nshp[k] = data.shape[k]

        idx=[0]*16
        idx[3]=min(Coil,nshp[3]-1)
        idx[4]=min(Map,nshp[4]-1)
        idx[13]=min(Slice,nshp[13]-1)
        idx[15]=min(Batch,nshp[15]-1)

        idx[0]=slice(None, None, -1)
        idx[1]=slice(None, None, None)

        dat=np.abs(data.reshape(nshp)[tuple(idx)])
        if 0 < np.max(dat):
          dat=dat/np.max(dat)

        axs.flatten()[i].imshow(dat,cmap="gray",vmin=Range[0], vmax=Range[1])
        axs.flatten()[i].set_title(wtitle[i])
    for ax in axs.flatten():
      ax.axis("off")
    
    plt.show()

  nshp = [1] * 16
  for file in files:
    data=cfl.readcfl(file)
    for i in range(len(data.shape)):
      nshp[i] = max(data.shape[i], nshp[i])
  
  interact(update,
           Range=widgets.FloatRangeSlider(value=[0.,1.], min=0, max=1.),
           Coil=widgets.IntSlider(min=0, max=nshp[3]-1, step=1, value=0),
           Map=widgets.IntSlider(min=0, max=nshp[4]-1, step=1, value=0),
           Slice=widgets.IntSlider(min=0, max=nshp[13]-1, step=1, value=0),
           Batch=widgets.IntSlider(min=0, max=nshp[15]-1, step=1, value=0))

# The Variational Network and MoDL with `BART reconet`

The `reconet` command implements physics based reconstruction networks solving the inverse SENSE problem.
We have implemented

> Variational Network<sup>1</sup>:
$$
x^{(i)} = x^{(i-1)}  - \lambda \nabla||Ax -b||^2 + Net(x^{(i-1)}, \Theta^{(i)} )
$$
> MoDL<sup>2</sup>:
$$
\begin{align}
z^{(i)} &= Net\left(x^{(i-1)}, \Theta \right)\\
x^{(i)} &= \mathrm{argmin}_x ||Ax -b||^2 + \lambda ||x - z^{(i)}||^2
\end{align}
$$

>Where
+ $A$ - MRI forward operator $\mathcal{PFC}$
    + $\mathcal{P}$ - Sampling pattern
    + $\mathcal{F}$ - Fourier transform
    + $\mathcal{C}$ - Coil sensitivity maps
+ $b$ - measured k-space data
+ $x^{(i)}$ - reconstruction after $i$ iterations
+ $x^{(0)}=A^Hb$ - initialization
+ $\Theta$ - Weights

>1: Hammernik, K. et al. (2018), [Learning a variational network for reconstruction of accelerated MRI data](https://doi.org/10.1002/mrm.26977). Magn. Reson. Med., 79: 3055-3071.

>2: Aggarwal, H. K. et al.(2019), [MoDL: Model-Based Deep Learning Architecture for Inverse Problems](https://doi.org/10.1109/TMI.2018.2865356). IEEE Trans. Med. Imag., 38(2): 394-405

To **train**, **evaluate** or **apply** unrolled networks, we provide the `bart reconet` command. Let us look at the help:

In [ ]:
%%bash

bart reconet -h

Usage: reconet [-t,--train] [-e,--eval] [-a,--apply] [-g,--gpu] [-l,--load <file>] [-b,--batch-size d] [-I,--iterations d] [-n,--normalize] [-N,--network ...] [--resnet-block ...] [--varnet-block ...] [--unet ...] [--data-consistency ...] [--initial-reco ...] [--shared-weights] [--no-shared-weights] [--shared-lambda] [--no-shared-lambda] [--rss-norm] [--trajectory <file>] [--pattern <file>] [--mask <file>] [--valid-data ...] [--train-loss ...] [--valid-loss ...] [-T,--train-algo ...] [--adam ...] [--iPALM ...] [--load-memory] [--lowmem] [--test] [--export-graph <string>] <kspace> <sensitivities> <weights> <ref/out> 

Trains or appplies a neural network for reconstruction.

-t,--train                   train reconet
-e,--eval                    evaluate reconet
-a,--apply                   apply reconet
-g,--gpu                     run on gpu
-l,--load <weights-init>     load weights for continuing training
-b,--batch-size d            size of mini batches
-I,--iterations d            n

## 3 Obtain k-Space Data

In this tutorial, we work with the *coronal_pd* data from the NYU machine learning dataset.
The data is availabel in the ismrmrd format at `mridata.org`.
We tried to match the UUIDs from `mridata.org` with the patient numbers from the Variational Network publication, however, that did not work for all patients.
Here, we restrict ot the patients which could be matched.

### 3.1 Defining Training and Validation Datasets

In the following cell, we define environment variables describing which patient should be used for the training dataset and which patient for validation.
Moreover, we create one `W`orking directory and an `A`rchive directory for the downloaded datasets to be reused.

In [ ]:
if "1" == os.environ["DEMO"]:

    # in DEMO mode, we use the same data for (short) training and validation
    os.environ['DAT']="pat_19"            # All data to be processed
    os.environ['TRN_DAT']="pat_19"        # Training dataset
    os.environ['VAL_DAT']="pat_19"        # Validation dataset
    os.environ['WDIR']=os.getcwd()+"/coronal_pd_demo/"
else:

    os.environ['TRN_DAT']="pat_2 pat_3 pat_4 pat_5 pat_6 pat_7 pat_8 pat_9 pat_10 pat_11 pat_12 pat_13 pat_14"
    os.environ['VAL_DAT']="pat_19"
    os.environ['DAT'] = os.environ['TRN_DAT'] + " " + os.environ['VAL_DAT']
    os.environ['WDIR']=os.getcwd()+"/coronal_pd/"

os.environ['ADIR']=os.getcwd()+"/archive/coronal_pd/"

!mkdir -p $ADIR
!mkdir -p $WDIR

### 3.2 Download Required Knee Data from mridata.org


In the following two cells, we download the datasets defined in the `DAT` variable if they are not already in the archive.

Afterwards, we use the `bart ismrmrd` tool to convert the ismrmrd format to the BART native *.cfl format. Be cautios, BART support for ismrmrd format is very limited and experimental!

Both cells perform the same operation, but they are split to provide live output in the first one.


In [ ]:
if not(os.path.isfile(os.environ["ADIR"]+"pat_19.hdr")):
    !wget -O $ADIR/pat_19.h5 http://mridata.org/download/993716e9-b5f0-4c30-a059-c003182d0f9c    # download data from mridata.org
    !bart ismrmrd --interleaved-siemens -o0 $ADIR/pat_19.h5 $ADIR/pat_19                         # transform to cfl

--2022-02-28 14:41:41--  http://mridata.org/download/993716e9-b5f0-4c30-a059-c003182d0f9c
Resolving mridata.org (mridata.org)... 35.160.223.32, 44.230.251.211
Connecting to mridata.org (mridata.org)|35.160.223.32|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://mridata-org-assets.s3.amazonaws.com/media/993716e9-b5f0-4c30-a059-c003182d0f9c.h5 [following]
--2022-02-28 14:41:41--  https://mridata-org-assets.s3.amazonaws.com/media/993716e9-b5f0-4c30-a059-c003182d0f9c.h5
Resolving mridata-org-assets.s3.amazonaws.com (mridata-org-assets.s3.amazonaws.com)... 52.218.216.18
Connecting to mridata-org-assets.s3.amazonaws.com (mridata-org-assets.s3.amazonaws.com)|52.218.216.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 934377520 (891M) [binary/octet-stream]
Saving to: ‘/content/bart-webinars/webinar6/archive/coronal_pd//pat_19.h5’

/content/bart-webin 100%[===================>] 891.09M  37.5MB/s    in 25s     

2022-02-28 14:42:06

In [ ]:
%%bash

UUIDs=(
1b996273-8d22-4f6e-a062-1e8cb4e9800b
3dafaac6-e40a-45c3-a285-c88944e72dab
662e8e76-9f0c-4f90-8fa0-85d1be9e68db
6b05c23b-a69d-44b9-a176-a0e3181fad57
70983198-5b4e-4081-91f4-0c461f7daebd
724570fe-a822-4faf-9835-8011e318f836
78d1ca02-a565-472d-bdc0-76450ade8cdf
7d1e0fb6-0a43-4548-980c-4f0cdb20c367
80c558dc-908f-4c64-84b0-e8e16da214c0
9270505a-8d77-4e43-ac43-0d9910b81510
993716e9-b5f0-4c30-a059-c003182d0f9c
9d3c581f-0b10-446f-88ef-1b3014e400d6
a0062756-d20b-4dc7-ba1a-76b5367a7c45
a0de6aae-7096-4cc7-bbb4-a4e4d159f680
b03f0bf5-200e-45b5-818e-e4a37142e2f5
c1fb122c-708c-4581-aab0-5b01f382946f
c734e0b0-a0dc-418a-ba68-44b158b00c16
cd255c11-ff09-4dd1-96e1-b7c5ed06f29c
dc05dd93-dd4c-478d-b6b5-79fb80095b73
e1be2ec9-1934-463a-bc92-cfcb5b00031a   
)

NAMEs=(
pat_5
pat_3
pat_18
pat_2
pat_17
pat_6
pat_11
pat_1
pat_14
pat_7
pat_19
pat_4
pat_12
pat_16
pat_13
pat_6
pat_8
pat_9
pat_9
pat_10
)

for d in $DAT
do
    if [[ ! -f $ADIR/${d}.hdr ]]
    then
        #we match the UUID to the patient number
        for i in ${!NAMEs[*]}
        do
            if [ ${NAMEs[i]} = ${d} ]
            then
                wget -O $ADIR/${d}.h5 -o /dev/null http://mridata.org/download/${UUIDs[i]}  # download data from mridata.org
                bart ismrmrd --interleaved-siemens -o0 $ADIR/${d}.h5 $ADIR/${d} >/dev/null  # transform to cfl
            fi
        done
    fi
done

### 3.3 Extract Slices

We extract 20 (or 5 in DEMO mode) slices of each k-space and copy the data to our working directory.

In [ ]:
%%bash

if [ $DEMO -eq 1 ]
then
    EXTRACT="20 25" #we extract 5 slices
else
    EXTRACT="10 30" #we extract 20 slices
fi

for d in $DAT
do
    bart extract 13 $EXTRACT $ADIR/${d} $WDIR/${d}_ksp_os
done

ls $WDIR

pat_19_ksp_os.cfl
pat_19_ksp_os.hdr


## 4 Estimating Coil Sensitivity Maps and Reference Reconstruction

We use the ESPIRiT method to estimate coil sensitivity maps from the k-space center. 
Further, we define the coil-combined reconstruction of the fully-sampled k-space data as reference



As reference reconstruction, we use the coil combined images.

We visualize the results

### 4.1 Estimating Coil Sensitivity Maps

ESPIRiT is implemented in BART by the `ecalib`command.
As the `ecalib`-command does not support a batch mode, we loop over the slices of each patient, extract them and run the `ecalib`-command on each of them.
Finally, the estimated coil sensitivity maps are stacked along the slice dimension.

In [ ]:
%%bash

TDIR=`mktemp -d 2>/dev/null || mktemp -d -t 'mytmpdir'`;
trap 'rm -rf "$TDIR"' EXIT

cd $TDIR

export OMP_NUM_THREADS=1

for pat in $DAT
do
  echo "estimate coils for ${pat}"
  pat=$WDIR/$pat #get absolute path
  
  COLS=""

  for i in $(seq 0 $(($(bart show -d 13 ${pat}_ksp_os)-1))) # loop over all slices of the patient
  do
    bart slice 13 $i ${pat}_ksp_os ksp_$i                   # extract the ith slice
    bart ecalib -a -m1 -r24 ksp_$i col_$i >/dev/null        # estimate coil sensitivity maps
                                                            # -> if "&" is appended, all slices are computes in parallel
    COLS+=" col_$i"                                         # append coil to list for joinint
  done

  # wait for ecalib if called in subprocess
  wait
  
  bart join 13 $COLS ${pat}_col_os                          # join all the slices of one patient
done

estimate coils for pat_19


### 4.2 Removing Frequency Oversampling

As we are only interested in the center 320 pixel along the frequency-encoding direction, we remove the frequency oversampling of the k-space data and coil sensitivity maps.

As a byproduct, we obtain the coil images (cim) of the reduced field of view.

In [ ]:
%%bash

#Remove frequency oversampling

TDIR=`mktemp -d 2>/dev/null || mktemp -d -t 'mytmpdir'`;
trap 'rm -rf "$TDIR"' EXIT
cd $TDIR

for pat in $DAT
do
  pat=$WDIR/$pat

  bart fft -i -u $(bart bitmask 0 1) ${pat}_ksp_os cim_os
  bart resize -c 0 320 cim_os ${pat}_cim
  bart fft -u  $(bart bitmask 0 1) ${pat}_cim ${pat}_ksp
  
  bart resize -c 0 320 ${pat}_col_os ${pat}_col
done

### 4.3 Coil-combined Reference

We combine the fully-sampled coil images using the `fmac` command to obtain the coil-combined reference and visulaize the results:

In [ ]:
%%bash

for pat in $DAT
do
    pat=$WDIR/$pat
    bart fmac -s$(bart bitmask 3) -C ${pat}_cim ${pat}_col ${pat}_ref
done


In [ ]:
plot([os.environ["WDIR"]+"pat_19_col", os.environ["WDIR"]+"pat_19_cim", os.environ["WDIR"]+"pat_19_ref"])

interactive(children=(FloatRangeSlider(value=(0.0, 1.0), description='Range', max=1.0), IntSlider(value=0, des…

## 5 Undersampling Pattern

We create a undersampling mask with a regular sampling pattern (R=4) and 24 auto callibration lines.
The script stacks ones and zeros, adds the AC-region and performs binary thresholding. (Python code would probably look nicer):

In [ ]:
%%bash

AC=24
R=4

READ=320
PHS1=332
PHS_OS=18

TDIR=`mktemp -d 2>/dev/null || mktemp -d -t 'mytmpdir'`;
trap 'rm -rf "$TDIR"' EXIT
cd $TDIR

bart ones 1 1 one
bart zeros 1 $((R-1)) zero
bart join 0 one zero pat_0
bart repmat 1 $((PHS1/R)) pat_0 pat_1
bart reshape $(bart bitmask 0 1) 1 $PHS1 pat_1 pat_2

bart ones 2 1 18 one
bart join 1 one pat_2 one pat_3

bart ones 2 1 $AC one
bart resize -c 1 $((PHS1+2*PHS_OS)) one ac

bart saxpy 1 ac pat_3 pat_4

bart threshold -B 0.5 pat_4 pat_5

bart repmat 0 $READ pat_5 $WDIR/pat

### 5.1 CG-SENSE Baseline

A subsampled k-space is created by multiplication of the k-space and the sampling pattern.
To get a feeling how much undersampling effects the reconstruction result, we create a zero-filled reconstruction and use `pics` toolfor a CG-SENSE baseline reconstruction (without any regularization). As for ESPIRiT, we loop over all slices and reconstruct them independently.

Finally, we visualize the results.

In [ ]:
%%bash

TDIR=`mktemp -d 2>/dev/null || mktemp -d -t 'mytmpdir'`;
trap 'rm -rf "$TDIR"' EXIT
cd $TDIR

for pat in $VAL_DAT
do
    pat=$WDIR/$pat

    bart fmac ${pat}_ksp $WDIR/pat ksp_ss

    CG=""
    
    for i in $(seq 0 $(($(bart show -d 13 ksp_ss)-1)))
    do
      bart slice 13 $i ksp_ss ksp
      bart slice 13 $i ${pat}_col col
      bart pics -p$WDIR/pat -l2 ksp col cg_$i > /dev/null

      CG+=" cg_$i"
    done

    bart join 13 $CG ${pat}_rec_cg

    bart fft -i -u $(bart bitmask 0 1) ksp_ss cim_ss
    bart fmac -s$(bart bitmask 3) -C cim_ss ${pat}_col ${pat}_rec_zf
done

In [ ]:
plot([os.environ["WDIR"]+"pat", os.environ["WDIR"]+"pat_19_rec_zf", os.environ["WDIR"]+"pat_19_rec_cg", os.environ["WDIR"]+"pat_19_ref"])

interactive(children=(FloatRangeSlider(value=(0.0, 1.0), description='Range', max=1.0), IntSlider(value=0, des…

## 6 Joining Training and Validation Datasets

We have prepared all data required to train a neural network with the `reconet` command. To pass the data to the `reconet` command, we join all patients of the training dataset to one file.


### 6.1 Define Variables for the Joined Datasets

For convincing access of the joined datasets, we store their file pathes in environment variables:

In [ ]:
os.environ["TRN_KSP"]=os.environ['WDIR']+"trn_ksp"
os.environ["TRN_COL"]=os.environ['WDIR']+"trn_col"
os.environ["TRN_REF"]=os.environ['WDIR']+"trn_ref"
os.environ["TRN_PAT"]=os.environ['WDIR']+"pat"

os.environ["VAL_KSP"]=os.environ['WDIR']+"val_ksp"
os.environ["VAL_COL"]=os.environ['WDIR']+"val_col"
os.environ["VAL_REF"]=os.environ['WDIR']+"val_ref"
os.environ["VAL_PAT"]=os.environ['WDIR']+"pat"

### 6.2 Join and Reshape

The **reconet** command follows the usual conventions of dimensions in BART defined in **src/misc/mri.h**. Independent datasets should be stacked along the **BATCH** dimension (15).

For training, we interpret the slices of the respective patients as independent datasets, i.e. we stack the data of all patients along the **SLICE** dimension and reshape the **SLICE** diemension to the **BATCH** dimension.

In [ ]:
%%bash

TDIR=`mktemp -d 2>/dev/null || mktemp -d -t 'mytmpdir'`;
trap 'rm -rf "$TDIR"' EXIT
cd $TDIR

DAT_KSP=""
DAT_COL=""
DAT_REF=""

for d in $TRN_DAT
do
DAT_KSP+=" ${WDIR}/${d}_ksp"
DAT_COL+=" ${WDIR}/${d}_col"
DAT_REF+=" ${WDIR}/${d}_ref"
done

bart join 13 $DAT_KSP tmp
bart reshape $(bart bitmask 13 15) 1 $(bart show -d13 tmp) tmp $TRN_KSP

bart join 13 $DAT_COL tmp
bart reshape $(bart bitmask 13 15) 1 $(bart show -d13 tmp) tmp $TRN_COL

bart join 13 $DAT_REF tmp
bart reshape $(bart bitmask 13 15) 1 $(bart show -d13 tmp) tmp $TRN_REF


In [ ]:
%%bash

TDIR=`mktemp -d 2>/dev/null || mktemp -d -t 'mytmpdir'`;
trap 'rm -rf "$TDIR"' EXIT
cd $TDIR

DAT_KSP=""
DAT_COL=""
DAT_REF=""

for d in $VAL_DAT
do
DAT_KSP+=" ${WDIR}/${d}_ksp"
DAT_COL+=" ${WDIR}/${d}_col"
DAT_REF+=" ${WDIR}/${d}_ref"
done

bart join 13 $DAT_KSP tmp
bart reshape $(bart bitmask 13 15) 1 $(bart show -d13 tmp) tmp $VAL_KSP

bart join 13 $DAT_COL tmp
bart reshape $(bart bitmask 13 15) 1 $(bart show -d13 tmp) tmp $VAL_COL

bart join 13 $DAT_REF tmp
bart reshape $(bart bitmask 13 15) 1 $(bart show -d13 tmp) tmp $VAL_REF

### 6.3 Inspect the Datasets

We check the dimensions of the constructed training dataset:

In [ ]:
%%bash

bart show -m $TRN_KSP
bart show -m $TRN_COL
bart show -m $TRN_REF

Type: complex float
Dimensions: 16
AoD:	320	368	1	15	1	1	1	1	1	1	1	1	1	1	1	5
Type: complex float
Dimensions: 16
AoD:	320	368	1	15	1	1	1	1	1	1	1	1	1	1	1	5
Type: complex float
Dimensions: 16
AoD:	320	368	1	1	1	1	1	1	1	1	1	1	1	1	1	5


And visualize them:

In [ ]:
plot([os.environ["TRN_KSP"], os.environ["TRN_COL"], os.environ["TRN_REF"]])

interactive(children=(FloatRangeSlider(value=(0.0, 1.0), description='Range', max=1.0), IntSlider(value=0, des…

## 7 A First Training - Basic Options

Finally, we have everything prepared to run the `reconet` command and train MoDL or the Variational Network.

### 7.1 The Variational Network 

Here, we describe each option we provide the `reconet` command to train the Variational network:

````
bart reconet              \
    --network=varnet -I10 \ #We train the Variational Network with 10 iterations
    --train               \ #Select train mode
    -b2                   \ #Batch size is 2
    --gpu                 \ #Obvious :)
    --normalize           \ #Normalize data such that max A^Hk=1
    --train-algo epochs=1 \ #One epoch
    --pattern=$TRN_PAT    \ #Pattern could be estimated from k-space 
    $TRN_KSP $TRN_COL $WGH $TRN_REF
````

The output of `reconet` in the training mode are the weights `$WGH`, which are provided afterwards in the inference mode to obtain a reconstruction.

To train the neural network, the `reconet`-command constructs an operator comparing the output of the network with the reference. Training the corresponds to minimizing the loss with respect to the weights $\theta$:

$$ \theta^* = \mathrm{argmin}_\theta \sum_i L\left(\text{ref}_i, \mathtt{Net}(A^Hb_i, \mathcal{C}_i, \mathcal{P}_i; \theta)\right) $$

The inputs and output od the loss operator are printed in the training log.

We execute the command:

In [ ]:
!bart reconet --network=varnet -I10 --train -b2 --gpu --normalize --train-algo epochs=1 --pattern=$TRN_PAT $TRN_KSP $TRN_COL wgh_varnet $TRN_REF

No training algorithm selected! Fallback to default settings.
Train Reconet
NN
inputs: 7
reference      [320 368   1   1   1   1   1   1   1   1   1   1   1   1   1   2 ]
adjoint        [320 368   1   1   1   1   1   1   1   1   1   1   1   1   1   2 ]
coil           [320 368   1  15   1   1   1   1   1   1   1   1   1   1   1   2 ]
psf            [  1 368   1   1   1   1   1   1   1   1   1   1   1   1   1   2 ]
lambda         [ 10 ]
conv           [ 24   1  11  11   1  10 ]
rbf            [ 24  31  10 ]
outputs: 1
mse mag        [  1 ]
#1->2/2; [======    ]; time: 0:00:08/0:00:08; loss: 2.867062e-03; l0: 2.008e-01; l1: 2.009e-01; l2: 2.009e-01; l3: 2.010e-01; l4: 2.011e-01; l5: 2.012e-01; l6: 2.013e-01; l7: 2.014e-01; l8: 2.016e-01; l9: 2.017e-01; L[4]=6.944e-01 L[5]=6.944e-01 L[6]=6.944e-01 


The trained weights are stored as complex floats in a *.cfl file. We have a look at its header:

In [ ]:
%%bash

cat wgh_varnet.hdr

# Dimensions
36490 
# SizesDimensions
1 6 3 
# MultiDimensions
10 
24 1 11 11 1 10 
24 31 10 
# Command
reconet --network=varnet -I10 --train -b2 --gpu --normalize --train-algo epochs=1 --pattern=/content/bart-webinars/webinar6/coronal_pd_demo/pat /content/bart-webinars/webinar6/coronal_pd_demo/trn_ksp /content/bart-webinars/webinar6/coronal_pd_demo/trn_col wgh_varnet /content/bart-webinars/webinar6/coronal_pd_demo/trn_ref 
# Files
 <>/content/bart-webinars/webinar6/coronal_pd_demo/trn_ref <>wgh_varnet </content/bart-webinars/webinar6/coronal_pd_demo/trn_col </content/bart-webinars/webinar6/coronal_pd_demo/trn_ksp </content/bart-webinars/webinar6/coronal_pd_demo/pat
# Creator
BART v0.7.00-357-gcf0baa7


### 7.2 MoDL

Similarly, we can train MoDL. We only command options differing from the Variational Network. The authors of MoDL propose to train their network in two steps:

````
bart reconet              \
    --network=modl -I1    \ #We initialize MoDL weights by training with one iteration
    --train               \
    -b2                   \
    --gpu --normalize     \
    --train-algo epochs=5 \ #We use some epochs such that the batch normalization can estimate floating mean/variance
    --pattern=$TRN_PAT    \
    $TRN_KSP $TRN_COL wgh_modl_one $TRN_REF

bart reconet
    --network=modl -I5    \
    --lowmem              \ #Reduce memory by checkpointing but double some computations
    --train               \
    -b2                   \
    --gpu --normalize     \
    --train-algo epochs=1 \
    -l wgh_modl_one       \ #We initialize with the weights from the first step
    --pattern=$TRN_PAT    \
    $TRN_KSP $TRN_COL wgh_modl $TRN_REF
````

In [ ]:
!bart reconet --network=modl -I1 --train -b2 --gpu --normalize --train-algo epochs=5 --pattern=$TRN_PAT $TRN_KSP $TRN_COL wgh_modl_one $TRN_REF
!bart reconet --network=modl -I5 --lowmem --train -b2 --gpu --normalize --train-algo epochs=1 -l wgh_modl_one --pattern=$TRN_PAT $TRN_KSP $TRN_COL wgh_modl $TRN_REF

No training algorithm selected! Fallback to default settings.
Train Reconet
NN
inputs: 15
reference      [320 368   1   1   1   1   1   1   1   1   1   1   1   1   1   2 ]
adjoint        [320 368   1   1   1   1   1   1   1   1   1   1   1   1   1   2 ]
coil           [320 368   1  15   1   1   1   1   1   1   1   1   1   1   1   2 ]
psf            [  1 368   1   1   1   1   1   1   1   1   1   1   1   1   1   2 ]
lambda         [  1 ]
conv_0         [ 32   1   3   3   1   1 ]
conv_i         [ 32  32   3   3   1   3   1 ]
conv_n         [  1  32   3   3   1   1 ]
bias_0         [ 32   1 ]
bias_i         [ 32   3   1 ]
bias_n         [  1   1 ]
gamma          [  1   1   1   1   1   1 ]
bn_0           [ 32   1   1   1   1   2   1 ]
bn_i           [ 32   1   1   1   1   2   3   1 ]
bn_n           [  1   1   1   1   1   2   1 ]
outputs: 4
mse            [  1 ]
bn_0           [ 32   1   1   1   1   2   1 ]
bn_i           [ 32   1   1   1   1   2   3   1 ]
bn_n           [  1   1   1   1   1

## 8 Applying the Networks

To apply the networks, we provide the `--apply` option instead of the `--train` options:

In [ ]:
%%bash

bart reconet --network=varnet -I10 --apply  --gpu --normalize --pattern=$VAL_PAT $VAL_KSP $VAL_COL wgh_varnet out_varnet > /dev/null
bart reconet --network=modl   -I5  --apply  --gpu --normalize --pattern=$VAL_PAT $VAL_KSP $VAL_COL wgh_modl   out_modl	 > /dev/null

In [ ]:
plot([os.environ["VAL_REF"], "out_varnet", "out_modl"])

interactive(children=(FloatRangeSlider(value=(0.0, 1.0), description='Range', max=1.0), IntSlider(value=0, des…

## 9 Extended Training

To obtain better results, we take some more time an trin for more epochs.
We can monitor the training by providing the validation dataset which is evaluated after each epoch. We use the `--valid-data` option:

### 9.1 The Variational Network

In [ ]:
%%bash

mkdir -p pretrained
cd pretrained

if [ $DEMO -eq "1" ]
then
    echo "Training skipped in DEMO mode" 
else
    bart reconet --network=varnet  --gpu --normalize                                     \
        --train -T epochs=30,batchgen-shuffle-data                                       \
        --valid-data pattern=${VAL_PAT},kspace=${VAL_KSP},coil=${VAL_COL},ref=${VAL_REF} \
        --pattern=$TRN_PAT $TRN_KSP $TRN_COL wgh_varnet $TRN_REF                         \
        >log_varnet.log
fi


cat log_varnet.log

Training skipped in DEMO mode
No training algorithm selected! Fallback to default settings.
Train Reconet
NN
inputs: 7
reference      [320 368   1   1   1   1   1   1   1   1   1   1   1   1   1  10 ]
adjoint        [320 368   1   1   1   1   1   1   1   1   1   1   1   1   1  10 ]
coil           [320 368   1  15   1   1   1   1   1   1   1   1   1   1   1  10 ]
psf            [  1 368   1   1   1   1   1   1   1   1   1   1   1   1   1  10 ]
lambda         [ 10 ]
conv           [ 24   1  11  11   1  10 ]
rbf            [ 24  31  10 ]
outputs: 1
mse mag        [  1 ]
#1->26/26; [==========]; time: 0:01:57/0:01:57; loss: 7.195710e-04; mse mag: 7.775e-04; mse: 1.012e-03; mean psnr: 3.167e+01; mean ssim: 8.640e-01; mad: 3.017e-02; mad mag: 1.840e-02; nmse: 8.303e-03; nmse mag: 6.379e-03; l0: 4.920e-01; l1: 5.011e-01; l2: 5.073e-01; l3: 5.140e-01; l4: 5.228e-01; l5: 5.285e-01; l6: 5.395e-01; l7: 5.576e-01; l8: 5.743e-01; l9: 6.001e-01; L[4]=8.735e-03 L[5]=8.735e-03 L[6]=8.735e-03 
#2->26/2

### 9.2 MoDL

In [ ]:
%%bash

mkdir -p pretrained
cd pretrained

if [ $DEMO -eq "1" ]
then
    echo "Training skipped in DEMO mode" 
else
    bart reconet --network=modl -I1                                      \
        --train -T e=50,batchgen-shuffle-data  --gpu --normalize         \
        --lowmem                                                         \
        --valid-data pattern=${VAL_PAT},kspace=${VAL_KSP},coil=${VAL_COL},ref=${VAL_REF} \
        --pattern=$TRN_PAT $TRN_KSP $TRN_COL wgh_modl_one $TRN_REF        \
        >log_modl.log

    bart reconet --network=modl -I5                                      \
        --train -T e=50,batchgen-shuffle-data  --gpu --normalize         \
        --lowmem -lwgh_modl_one                                      \
        --valid-data pattern=${VAL_PAT},kspace=${VAL_KSP},coil=${VAL_COL},ref=${VAL_REF} \
        --pattern=$TRN_PAT $TRN_KSP $TRN_COL wgh_modl $TRN_REF            \
        >>log_modl.log
fi

cat log_modl.log

Training skipped in DEMO mode
No training algorithm selected! Fallback to default settings.
Train Reconet
NN
inputs: 15
reference      [320 368   1   1   1   1   1   1   1   1   1   1   1   1   1  10 ]
adjoint        [320 368   1   1   1   1   1   1   1   1   1   1   1   1   1  10 ]
coil           [320 368   1  15   1   1   1   1   1   1   1   1   1   1   1  10 ]
psf            [  1 368   1   1   1   1   1   1   1   1   1   1   1   1   1  10 ]
lambda         [  1 ]
conv_0         [ 32   1   3   3   1   1 ]
conv_i         [ 32  32   3   3   1   3   1 ]
conv_n         [  1  32   3   3   1   1 ]
bias_0         [ 32   1 ]
bias_i         [ 32   3   1 ]
bias_n         [  1   1 ]
gamma          [  1   1   1   1   1   1 ]
bn_0           [ 32   1   1   1   1   2   1 ]
bn_i           [ 32   1   1   1   1   2   3   1 ]
bn_n           [  1   1   1   1   1   2   1 ]
outputs: 4
mse            [  1 ]
bn_0           [ 32   1   1   1   1   2   1 ]
bn_i           [ 32   1   1   1   1   2   3   1 ]
bn_n 

### 9.3 Visualization

In [ ]:
%%bash

bart reconet --network=varnet -I10 --apply  --gpu --normalize --pattern=$VAL_PAT $VAL_KSP $VAL_COL pretrained/wgh_varnet out_varnet > /dev/null
bart reconet --network=modl   -I5  --apply  --gpu --normalize --pattern=$VAL_PAT $VAL_KSP $VAL_COL pretrained/wgh_modl   out_modl   > /dev/null

In [ ]:
plot([os.environ["VAL_REF"], "out_varnet", "out_modl"])

interactive(children=(FloatRangeSlider(value=(0.0, 1.0), description='Range', max=1.0), IntSlider(value=0, des…

## 10 Evaluation

To obtain a quantitative evaluation of the trained networks, we can pass the `--eval` option to first apply the trained network and compare with a refernce.

In [ ]:
%%bash

bart reconet --network=varnet -I10 --eval  --gpu --normalize --pattern=$VAL_PAT $VAL_KSP $VAL_COL pretrained/wgh_varnet $VAL_REF

Apply RecoNet
NN
inputs: 6
adjoint        [320 368   1   1   1   1   1   1   1   1   1   1   1   1   1   1 ]
coil           [320 368   1  15   1   1   1   1   1   1   1   1   1   1   1   1 ]
psf            [  1 368   1   1   1   1   1   1   1   1   1   1   1   1   1   1 ]
lambda         [ 10 ]
conv           [ 24   1  11  11   1  10 ]
rbf            [ 24  31  10 ]
outputs: 1
INDEX 0        [320 368   1   1   1   1   1   1   1   1   1   1   1   1   1   1 ]
mse mag: 2.388779e-11
mse: 3.659311e-11
mean psnr: 3.644878e+01
mean ssim: 9.202690e-01
mad: 6.392077e-06
mad mag: 3.585330e-06
nmse: 2.731247e-03
nmse mag: 1.771816e-03


In [ ]:
%%bash

bart reconet --network=modl   -I5  --eval  --gpu --normalize --pattern=$VAL_PAT $VAL_KSP $VAL_COL pretrained/wgh_modl $VAL_REF

Apply RecoNet
NN
inputs: 14
adjoint        [320 368   1   1   1   1   1   1   1   1   1   1   1   1   1   1 ]
coil           [320 368   1  15   1   1   1   1   1   1   1   1   1   1   1   1 ]
psf            [  1 368   1   1   1   1   1   1   1   1   1   1   1   1   1   1 ]
lambda         [  1 ]
conv_0         [ 32   1   3   3   1   1 ]
conv_i         [ 32  32   3   3   1   3   1 ]
conv_n         [  1  32   3   3   1   1 ]
bias_0         [ 32   1 ]
bias_i         [ 32   3   1 ]
bias_n         [  1   1 ]
gamma          [  1   1   1   1   1   5 ]
bn_0           [ 32   1   1   1   1   2   5 ]
bn_i           [ 32   1   1   1   1   2   3   5 ]
bn_n           [  1   1   1   1   1   2   5 ]
outputs: 4
INDEX 0        [320 368   1   1   1   1   1   1   1   1   1   1   1   1   1   1 ]
bn_0           [ 32   1   1   1   1   2   5 ]
bn_i           [ 32   1   1   1   1   2   3   5 ]
bn_n           [  1   1   1   1   1   2   5 ]
mse mag: 2.181674e-11
mse: 3.326093e-11
mean psnr: 3.679855e+01
mean ssim

# Extensions: Some Options

The BART implementation of `reconet` implements many options for the training process. A list of all options with a short description is available by the help function.

In this section, we present some of them:

### 11.1 Training Algorithm

The training algorithm can be configured with the `--train-algo` option. We have already used this option to configure the number of epochs and to shuffle the training data after each epoch:


In [ ]:
%%bash

bart reconet --train-algo h

Usage of sub-option: -T,--train-algo [r=f,learning-rate=f],[e=d,epochs=d],[sgd],[adadelta],[adam],[ipalm],[clip-norm=f],[clip-value=f],[weight-decay=f],[learning-rate-min=f],[cosine-annealing-mod=d],[export-history=<file>],[dump-mod=d],[batchnorm-momentum=f],[batchgen-same],[batchgen-shuffle-batches],[batchgen-shuffle-data],[batchgen-draw-data],[batchgen-seed=d],[average-loss]

Sub-options: configure general training parmeters

r,learning-rate=f           learning rate
e,epochs=d                  number of epochs to train
sgd                         select stochastic gradient descent
adadelta                    select AdaDelta
adam                        select Adam
ipalm                       select iPALM
clip-norm=f                 clip norm of gradients
clip-value=f                clip value of gradients
weight-decay=f              reduce weights by 1 / (1 + lr*f) after each update
learning-rate-min=f         minimum learnin rate (used for scheduled learning rate)
cosine-annealing-m

### 11.2 Loss configuration

The training loss can be configured using the `--train-loss` option.
Different losses can be combined by setting their weighting. By default, MoDL uses `mse` and the Variational Network uses `mse-magnitude`.

If coil images are provided as reference data, losses acting on magnitude images automatically apply on the rss of the coil images.

Note that not all losses are suitable for images.

In [ ]:
%%bash

bart reconet --train-loss h

Usage of sub-option: --train-loss [mse=f],[mad=f],[nmse=f],[mse-magnitude=f],[mad-magnitude=f],[nmse-magnitude=f],[ssim=f],[cce=f],[wcce=f],[dice-0=f],[dice-1=f],[dice-2=f]

Sub-options: configure the training loss

mse=weighting               weighting for mean squared error
mad=weighting               weighting for mean absolute difference
nmse=weighting              weighting for normalized mean squared error
mse-magnitude=weighting     weighting for mean squared error (rss)
mad-magnitude=weighting     weighting for mean absolute difference (rss)
nmse-magnitude=weighting    weighting for normalized mean squared error (rss)
ssim=weighting              weighting for structural similarity index measure (rss)
cce=weighting               weighting for categorical cross entropy
wcce=weighting              weighting for weighted categorical cross entropy
dice-0=weighting            weighting for unbalanced dice loss
dice-1=weighting            weighting for dice loss weighted with inverse 

### 11.3 Initial Reconstruction / Data-Consistency

The data-consistency term can be configured using the respective option. By default, MoDL uses `proximal-mapping` and the Variational Network uses `gradient-step`.

By default, both networks are initialized with a zero-filled (i.e. adjoint) reconstruction. We can change that to a CG-SENSE reconstruction `--initial-reco tickhonov`.

In [ ]:
%%bash

bart reconet --data-consistency h
bart reconet --initial-reco h

Usage of sub-option: --data-consistency [fix-lambda=f],[lambda-init=f],[gradient-step],[gradient-max-eigen],[proximal-mapping],[max-cg-iter=d]

Sub-options: configure data-consistency method

fix-lambda=f          fix lambda to specified value (-1 means train lambda)
lambda-init=f         initialize lambda with specified value
gradient-step         use gradient steps for data-consistency
gradient-max-eigen    scale stepsize by inverse max eigen value of A^HA
proximal-mapping      use proximal mapping for data-consistency
max-cg-iter=d         number of cg steps for proximal mapping
h                     help
Usage of sub-option: --initial-reco [tickhonov],[max-cg-iter=d],[fix-lambda=f],[lambda-init=f]

Sub-options: configure initialization

tickhonov        init network with Tickhonov regularized reconstruction instead of adjoint reconstruction
max-cg-iter=d    number of cg steps for Tickhonov regularized reconstruction
fix-lambda=f     fix lambda to specified value (-1 means train lam

### 11.4 Network Parameter

Further, we can configure the network parameter of the residual block in MoDL or the variational block in the Varaiational Network.

In [ ]:
%%bash

bart reconet --varnet-block h
bart reconet --resnet-block h

Usage of sub-option: --varnet-block [W=d,basis=d],[F=d,filters=d],[X=d,filter-x=d],[Y=d,filter-y=d],[Z=d,filter-z=d]

Sub-options: configure variational block

W,basis=d       number of basis functions (default: 31)
F,filters=d     number of filters in residual block (default: 24)
X,filter-x=d    filter size in x-dimension (default: 11)
Y,filter-y=d    filter size in y-dimension (default: 11)
Z,filter-z=d    filter size in z-dimension (default: 1)
h               help
Usage of sub-option: --resnet-block [L=d,layers=d],[F=d,filters=d],[X=d,filter-x=d],[Y=d,filter-y=d],[Z=d,filter-z=d],[no-batch-normalization],[no-bias]

Sub-options: configure residual block

L,layers=d                number of layers in residual block (default: 5)
F,filters=d               number of filters in residual block (default: 32)
X,filter-x=d              filter size in x-dimension (default: 3)
Y,filter-y=d              filter size in y-dimension (default: 3)
Z,filter-z=d              filter size in z-dimension

### 11.5 Retrain MoDL
We can retrain MoDL using some different options

In [ ]:
%%bash

mkdir -p pretrained
cd pretrained

if [ $DEMO -eq "1" ]
then
    echo "Training skipped in DEMO mode" 
else
    bart reconet --network=modl -I1                                      \
        --train -T e=50,batchgen-shuffle-data  --gpu --normalize         \
        --train -T learning-rate=0.0001                                  \
        --train-loss mse=1.,mad-magnitude=0.01                           \
        --initial-reco tickhonov                                         \
        --lowmem                                                         \
        --valid-data pattern=${VAL_PAT},kspace=${VAL_KSP},coil=${VAL_COL},ref=${VAL_REF} \
        --pattern=$TRN_PAT $TRN_KSP $TRN_COL wgh_modl_one_mad $TRN_REF   \
        >log_modl_mad.log

    bart reconet --network=modl -I5                                      \
        --train -T e=50,batchgen-shuffle-data  --gpu --normalize         \
        --train -T learning-rate=0.0001                                  \
        --train-loss mse=1.,mad-magnitude=0.01                           \
        --initial-reco tickhonov                                         \
        --lowmem -lwgh_modl_one_mad                                      \
        --valid-data pattern=${VAL_PAT},kspace=${VAL_KSP},coil=${VAL_COL},ref=${VAL_REF} \
        --pattern=$TRN_PAT $TRN_KSP $TRN_COL wgh_modl_mad $TRN_REF       \
        >>log_modl_mad.log
fi

cat log_modl_mad.log

Training skipped in DEMO mode
No training algorithm selected! Fallback to default settings.
Train Reconet
NN
inputs: 15
reference      [320 368   1   1   1   1   1   1   1   1   1   1   1   1   1  10 ]
adjoint        [320 368   1   1   1   1   1   1   1   1   1   1   1   1   1  10 ]
coil           [320 368   1  15   1   1   1   1   1   1   1   1   1   1   1  10 ]
psf            [  1 368   1   1   1   1   1   1   1   1   1   1   1   1   1  10 ]
lambda         [  1 ]
conv_0         [ 32   1   3   3   1   1 ]
conv_i         [ 32  32   3   3   1   3   1 ]
conv_n         [  1  32   3   3   1   1 ]
bias_0         [ 32   1 ]
bias_i         [ 32   3   1 ]
bias_n         [  1   1 ]
gamma          [  1   1   1   1   1   1 ]
bn_0           [ 32   1   1   1   1   2   1 ]
bn_i           [ 32   1   1   1   1   2   3   1 ]
bn_n           [  1   1   1   1   1   2   1 ]
outputs: 4
mse + mad mag  [  1 ]
bn_0           [ 32   1   1   1   1   2   1 ]
bn_i           [ 32   1   1   1   1   2   3   1 ]
bn_n 

In [ ]:
%%bash

bart reconet --network=modl                          -I5 --eval --gpu --normalize --pattern=$VAL_PAT $VAL_KSP $VAL_COL pretrained/wgh_modl $VAL_REF
bart reconet --network=modl --initial-reco tickhonov -I5 --eval --gpu --normalize --pattern=$VAL_PAT $VAL_KSP $VAL_COL pretrained/wgh_modl_mad $VAL_REF

Apply RecoNet
NN
inputs: 14
adjoint        [320 368   1   1   1   1   1   1   1   1   1   1   1   1   1   1 ]
coil           [320 368   1  15   1   1   1   1   1   1   1   1   1   1   1   1 ]
psf            [  1 368   1   1   1   1   1   1   1   1   1   1   1   1   1   1 ]
lambda         [  1 ]
conv_0         [ 32   1   3   3   1   1 ]
conv_i         [ 32  32   3   3   1   3   1 ]
conv_n         [  1  32   3   3   1   1 ]
bias_0         [ 32   1 ]
bias_i         [ 32   3   1 ]
bias_n         [  1   1 ]
gamma          [  1   1   1   1   1   5 ]
bn_0           [ 32   1   1   1   1   2   5 ]
bn_i           [ 32   1   1   1   1   2   3   5 ]
bn_n           [  1   1   1   1   1   2   5 ]
outputs: 4
INDEX 0        [320 368   1   1   1   1   1   1   1   1   1   1   1   1   1   1 ]
bn_0           [ 32   1   1   1   1   2   5 ]
bn_i           [ 32   1   1   1   1   2   3   5 ]
bn_n           [  1   1   1   1   1   2   5 ]
mse mag: 2.181674e-11
mse: 3.326093e-11
mean psnr: 3.679855e+01
mean ssim

In [ ]:
%%bash

bart reconet --network=modl -I5  --apply  --gpu --normalize --pattern=$VAL_PAT $VAL_KSP $VAL_COL pretrained/wgh_modl_mad out_modl_mad > /dev/null

In [ ]:
plot([os.environ["VAL_REF"], "out_modl_mad", "out_modl"])

interactive(children=(FloatRangeSlider(value=(0.0, 1.0), description='Range', max=1.0), IntSlider(value=0, des…

# Extensions: Non-Cartesian Trajectories

Finally, we present a reconstruction using non-Cartesian trajectories.
The trajectory can be passed with the `--trajectory` option to the reconet command.

In this example, we simulate non-Cartesian k-space data based on the fully-sampled coil images:

## 12.1 Simulate Non-Cartesian Dataset and Generate Training Data

In [ ]:
os.environ["TRJ"]=os.environ['WDIR']+"trj"

os.environ["TRN_KSP"]=os.environ['WDIR']+"trn_ksp_rad"
os.environ["VAL_KSP"]=os.environ['WDIR']+"val_ksp_rad"


In [ ]:
%%bash

bart traj -x 368 -y 30 -r $TRJ

for pat in $DAT
do 
    bart nufft $TRJ $WDIR/${pat}_cim $WDIR/${pat}_ksp_rad
done

Done.


In [ ]:
%%bash

TDIR=`mktemp -d 2>/dev/null || mktemp -d -t 'mytmpdir'`;
trap 'rm -rf "$TDIR"' EXIT
cd $TDIR

DAT_KSP=""

for d in $TRN_DAT
do
DAT_KSP+=" ${WDIR}/${d}_ksp_rad"
done

bart join 13 $DAT_KSP tmp
bart reshape $(bart bitmask 13 15) 1 $(bart show -d13 tmp) tmp $TRN_KSP

DAT_KSP=""

for d in $VAL_DAT
do
DAT_KSP+=" ${WDIR}/${d}_ksp_rad"
done

bart join 13 $DAT_KSP tmp
bart reshape $(bart bitmask 13 15) 1 $(bart show -d13 tmp) tmp $VAL_KSP

## 12.2 Training MoDL

To train MoDL, we use again the `--initial-reco tickhonov` option to initialize the network with the CG-SENSE reconstruction. We further increase the number of CG iterations and fix the regularization of the initial reconstruction to be not trained. Thus, the data is normalized such that the maximum magnitude of the CG-SENSE reconstruction is one.


In [ ]:
%%bash

if [ $DEMO -eq "1" ]
then
    echo Skipped
else

    bart reconet --network=modl -I1                                      \
        --train -T e=50,batchgen-shuffle-data,r=0.01  --gpu --normalize  \
        --initial-reco tickhonov,fix-lambda=0.1,max-cg-iter=30           \
        --data-consistency max-cg-iter=30                                \
        --valid-data trajectory=${TRJ},kspace=${VAL_KSP},coil=${VAL_COL},ref=${VAL_REF} \
        --trajectory=${TRJ} $TRN_KSP $TRN_COL pretrained/wgh_modl_one_rad $TRN_REF      \
        >pretrained/log_modl_rad.log

    bart reconet --network=modl -I5                                      \
        --train -T e=50,batchgen-shuffle-data  --gpu --normalize         \
        --lowmem -lpretrained/wgh_modl_one_rad                           \
        --initial-reco tickhonov,fix-lambda=0.1,max-cg-iter=30           \
        --data-consistency max-cg-iter=30                                \
        --valid-data trajectory=${TRJ},kspace=${VAL_KSP},coil=${VAL_COL},ref=${VAL_REF} \
        --trajectory=$TRJ $TRN_KSP $TRN_COL pretrained/wgh_modl_rad $TRN_REF            \
        >>pretrained/log_modl_rad.log

fi
cat pretrained/log_modl_rad.log

Skipped
No training algorithm selected! Fallback to default settings.
Train Reconet
NN
inputs: 15
reference      [320 368   1   1   1   1   1   1   1   1   1   1   1   1   1  10 ]
adjoint        [320 368   1   1   1   1   1   1   1   1   1   1   1   1   1  10 ]
coil           [320 368   1  15   1   1   1   1   1   1   1   1   1   1   1  10 ]
psf            [320 368   1   1   1   1   1   1   1   1   1   1   1   1   1  10   4 ]
lambda         [  1 ]
conv_0         [ 32   1   3   3   1   1 ]
conv_i         [ 32  32   3   3   1   3   1 ]
conv_n         [  1  32   3   3   1   1 ]
bias_0         [ 32   1 ]
bias_i         [ 32   3   1 ]
bias_n         [  1   1 ]
gamma          [  1   1   1   1   1   1 ]
bn_0           [ 32   1   1   1   1   2   1 ]
bn_i           [ 32   1   1   1   1   2   3   1 ]
bn_n           [  1   1   1   1   1   2   1 ]
outputs: 4
mse            [  1 ]
bn_0           [ 32   1   1   1   1   2   1 ]
bn_i           [ 32   1   1   1   1   2   3   1 ]
bn_n           [  1   1

## 12.3 L1-Wavelet Baseline Reconstruction

For reference, we perform a l1-wavelet baseline reconstruction.

In [ ]:
%%bash

TDIR=`mktemp -d 2>/dev/null || mktemp -d -t 'mytmpdir'`;
trap 'rm -rf "$TDIR"' EXIT
cd $TDIR

for pat in $VAL_DAT
do
    pat=$WDIR/$pat

    CG=""
    
    for i in $(seq 0 $(($(bart show -d 13 ${pat}_ksp_rad)-1)))
    do
      bart slice 13 $i ${pat}_ksp_rad ksp
      bart slice 13 $i ${pat}_col col
      bart pics -r0.0002 -l1 -e -i100 -t$TRJ ksp col cg_$i > /dev/null

      CG+=" cg_$i"
    done

    bart join 13 $CG ${pat}_rec_rad_l1
done

## 12.4 Apply Non-Cartesian MoDL

Finally, we apply the non-Cartesian MoDL and visualize the reconstruction

In [ ]:
%%bash

bart reconet --network=modl -I5 --apply --gpu --normalize                                     \
    --initial-reco tickhonov,fix-lambda=0.1,max-cg-iter=30 --data-consistency max-cg-iter=30  \
    --trajectory=$TRJ $VAL_KSP $VAL_COL pretrained/wgh_modl_rad out_noncart > /dev/null

In [ ]:
plot([os.environ["WDIR"]+"pat_19_ref", os.environ["WDIR"]+"pat_19_rec_rad_l1", "out_noncart"])

interactive(children=(FloatRangeSlider(value=(0.0, 1.0), description='Range', max=1.0), IntSlider(value=0, des…

# Thank you for your attention